In [5]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import Table, func
from dateutil.relativedelta import relativedelta
from datetime import datetime
from flask_mail import Mail, Message
from Models.helper import u_id_generator, f_id_generator, r_id_generator
# from Models.model import User, Film, Review, Review_Like, Review_Dislike
from datetime import datetime
import bcrypt
import pandas as pd

In [2]:
app = Flask(__name__)

db_config = {
    'SQLALCHEMY_DATABASE_URI' : 'sqlite:///Database/doubi_database.db',
    'SQLALCHEMY_TRACK_MODIFICATIONS' : False
}

app.config.update(db_config)
db = SQLAlchemy(app)

In [3]:
followers = Table('followers', db.metadata,
    db.Column('followed_id', db.String(32), db.ForeignKey('user.u_id')),
    db.Column('follower_id', db.String(32), db.ForeignKey('user.u_id'))
)

blocked_users = Table('blocked_users', db.metadata,
    db.Column('blocked_id', db.String(32), db.ForeignKey('user.u_id')),
    db.Column('blocker_id', db.String(32), db.ForeignKey('user.u_id')),
)

users_wish_film = Table('users_wish_film', db.metadata,
    db.Column('user_id', db.String(32), db.ForeignKey('user.u_id')),
    db.Column('film_id', db.String(32), db.ForeignKey('film.f_id'))
)


class User(db.Model):
    __tablename__ = 'user'
    
    u_id = db.Column(db.String(32), primary_key=True, nullable=False, unique=True, default=u_id_generator)
    username = db.Column(db.String(80), nullable=False, unique=True)
    password_hash = db.Column(db.Text, nullable=False)
    email = db.Column(db.String(80), nullable=False, unique=True)
    url_photo = db.Column(db.Text, nullable=True)
    is_admin = db.Column(db.Boolean, nullable=False, default=False)
    is_blocked = db.Column(db.Boolean, nullable=False, default=False)

    created_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)
    updated_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow, onupdate=datetime.utcnow)

    followed = db.relationship('User', 
                                secondary=followers,
                                primaryjoin=(followers.c.follower_id == u_id),
                                secondaryjoin=(followers.c.followed_id == u_id),
                                backref=db.backref('followers', lazy='dynamic'),
                                lazy='dynamic')

    blocked = db.relationship('User',
                                secondary=blocked_users,
                                primaryjoin=(blocked_users.c.blocker_id == u_id),
                                secondaryjoin=(blocked_users.c.blocked_id == u_id),
                                backref=db.backref('blockers', lazy='dynamic'),
                                lazy='dynamic')
    
    reviews = db.relationship('Review', backref='user', lazy='dynamic')

    review_likes = db.relationship('Review_Like', backref='user', lazy='dynamic')
    review_dislikes = db.relationship('Review_Dislike', backref='user', lazy='dynamic')
    
    wish = db.relationship('Film', secondary=users_wish_film, backref='user', lazy='dynamic')
    
    @property
    def password(self):
        raise AttributeError('password is not a readable attribute')

    @password.setter
    def password(self, password):
        salt = bcrypt.gensalt()
        self.password_hash = bcrypt.hashpw(password.encode('utf-8'), salt)
    
    def verify_password(self, password):
        return bcrypt.checkpw(password.encode('utf-8'), self.password_hash)
    
    def __repr__(self):
        return '<User %r>' % self.username


    

class Film(db.Model):
    __tablename__ = 'film'

    f_id = db.Column(db.String(32), primary_key=True, nullable=False, unique=True, default=f_id_generator)
    title = db.Column(db.String(80), nullable=False)
    genre = db.Column(db.String(80), nullable=False)
    year = db.Column(db.Integer, nullable=True)
    run_time = db.Column(db.String(16), nullable=True)
    rating_imdb = db.Column(db.Float, nullable=True)
    overview = db.Column(db.String(500), nullable=True)
    director = db.Column(db.String(80), nullable=True)
    actor = db.Column(db.String(200), nullable=True)
    url_poster = db.Column(db.Text, nullable=True)

    created_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)
    updated_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow, onupdate=datetime.utcnow)

    reviews = db.relationship('Review', backref='film', lazy='dynamic')
    
    wish_by = db.relationship('User', secondary=users_wish_film, backref='film', lazy='dynamic')
    
    def rating(self):
        reviews = self.reviews.all()
        if len(reviews) == 0:
            return 0
        else:
            return sum(review.rating for review in reviews) / len(reviews)

    
    def __repr__(self):
        return '<Film %r>' % self.title
    


class Review(db.Model):
    __tablename__ = 'review'

    r_id = db.Column(db.String(32), primary_key=True, nullable=False, unique=True, default=r_id_generator)
    u_id = db.Column(db.String(32), db.ForeignKey('user.u_id'), nullable=False)
    f_id = db.Column(db.String(32), db.ForeignKey('film.f_id'), nullable=False)
    content = db.Column(db.String(500), nullable=True)
    rating = db.Column(db.Integer, nullable=False)
    bad_word = db.Column(db.Boolean, nullable=False, default=False)

    created_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)

    likes = db.relationship('Review_Like', backref='review', lazy='dynamic')
    dislikes = db.relationship('Review_Dislike', backref='review', lazy='dynamic')


class Review_Like(db.Model):
    __tablename__ = 'review_like'

    r_id = db.Column(db.String(32), db.ForeignKey('review.r_id'), primary_key=True, nullable=False)
    u_id = db.Column(db.String(32), db.ForeignKey('user.u_id'), primary_key=True, nullable=False)

    created_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)


class Review_Dislike(db.Model):
    __tablename__ = 'review_dislike'

    r_id = db.Column(db.String(32), db.ForeignKey('review.r_id'), primary_key=True, nullable=False)
    u_id = db.Column(db.String(32), db.ForeignKey('user.u_id'), primary_key=True, nullable=False)

    created_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)

In [4]:
# from sqlite3 import connect
# conn = connect(app.root_path + '/Database/doubi_database_old.db')
# c = conn.cursor()

# c.execute('SELECT * FROM film')
# films = c.fetchall()
# print(films)
db.create_all()

In [43]:
from sqlite3 import connect
# conn = connect(app.root_path + '/Database/doubi_database_old.db')
conn = connect("D:/Workspace/capstone-project-9900-t16p-ahduiduidui/back-end/IMDB_Movies_2021.db")
c = conn.cursor()

c.execute('SELECT * FROM reviews')
reviews = c.fetchall()

In [27]:
reviews[0]

(1,
 "I don't get all the terrible reviews for this movie. Especially because lots of people talk incredible bad of the special effects. It's not Oscar worthy, but is not terrible. It's quite good for such type of movie.It lacks some quality on the script, but which shark movie doesn't?There's some other little mistakes, but overall is watchable. If you are a fan of this genre, o recommend you check it out this one.Ignore the below 4 scores. It's unfair for those who work in this project.",
 5,
 'margarida-44311',
 ' Not Bad\n')

In [44]:
reviews = [[reviews[i][1], reviews[i][2]] for i in range(len(reviews))]


In [51]:
import random
for i in range(len(reviews)):
    if type(reviews[i][1]) != int:
        reviews[i][1] = random.randint(1, 10)

In [52]:
reviews = [[reviews[i][0], reviews[i][1]//2] for i in range(len(reviews))]
reviews[:5]

[["I don't get all the terrible reviews for this movie. Especially because lots of people talk incredible bad of the special effects. It's not Oscar worthy, but is not terrible. It's quite good for such type of movie.It lacks some quality on the script, but which shark movie doesn't?There's some other little mistakes, but overall is watchable. If you are a fan of this genre, o recommend you check it out this one.Ignore the below 4 scores. It's unfair for those who work in this project.",
  2],
 ["I cannot believe anyone could give this film less than a 6 ,I gave it an 8 because I thought it was well acted great filming and an exciting story, c'mon man what's with all the negativity.",
  4],
 ["Great White is not the worst way to spend 90 minutes but it does have issues. In the film seaplane tour operators Charlie and Kaz with their cook Benny take out a wealthy couple Joji and Michelle to Impervious Reef. There they find a victim of a shark attack and head back. On the way back they se

In [58]:
users = User.query.all()
films = Film.query.all()

In [65]:
bad_words = pd.read_csv('D:/Users/zzy80/Desktop/bad-words.csv')

# change bad_word to list
bad_words = bad_words.values.tolist()
bad_words = [word[0] for word in bad_words]
# bad_word

In [ ]:
for i, review in enumerate(reviews):
    print("checking review: ", i)
    for bad_word in bad_words:
        if bad_word in review[0].lower().split():
            print("review %d contains bad word %s" % (i, bad_word))

In [88]:
num_of_reviews = []
for i in range(10):
    num_of_reviews.append(random.randint(1, 300))
    
num_of_reviews = sorted(num_of_reviews, reverse=True)
print(num_of_reviews)

for i in range(990):
    num_of_reviews.append(2)

[256, 253, 209, 165, 165, 162, 80, 68, 51, 22]


In [ ]:
while(sum(num_of_reviews) < len(reviews)):
    num_of_reviews[random.randint(0, 999)] += 1
    
print(num_of_reviews)

In [ ]:
num_of_like = [random.randint(50, 500) for i in range(len(users))]
num_of_dislike = [random.randint(0, 200) for i in range(len(users))]

print(num_of_like)
print(num_of_dislike)

In [ ]:
db.session.rollback()
for i, user in enumerate(users):
    print("%d - %s" % (i, user.username))
    review_like = random.sample(reviews, num_of_like[i])
    review_dislike = random.sample(list(set(reviews)-set(review_like)), num_of_dislike[i])
    for review in review_like:
        review_like_obj = Review_Like(r_id=review.r_id, u_id=user.u_id)
        db.session.add(review_like_obj)
    for review in review_dislike:
        review_dislike_obj = Review_Dislike(r_id=review.r_id, u_id=user.u_id)
        db.session.add(review_dislike_obj)

db.session.commit()

In [ ]:
db.session.rollback()

for i, user in enumerate(users):
    print("%d - %s" % (i, user.username))
    follow = random.sample(users, 5)
    while(user in follow):
        follow.remove(user)
        follow.append(random.choice(users))
    blocked = random.sample(users, 5)
    while(user in blocked):
        blocked.remove(user)
        blocked.append(random.choice(users))
    
    user.followed = follow
    user.blocked = blocked
db.session.commit()

In [149]:
db.session.rollback()
for i, user in enumerate(users):
    print("%d - %s" % (i, user.username))
    wish = random.sample(films, 2)
    user.wish = wish
db.session.commit()

0 - Aniruddha
1 - Weisz
2 - Gurira
3 - Boman
4 - Merlant
5 - Bernhard
6 - Mia
7 - Julie
8 - Bauer
9 - Lina
10 - Dormer
11 - Swinton
12 - Irani
13 - Prateik
14 - Poppe
15 - Dougnac
16 - Squires
17 - Solveig
18 - Campbell
19 - Jory
20 - Chong
21 - Kishimoto
22 - Inoue
23 - Rex
24 - Stiller
25 - Tilda
26 - Iko
27 - Anand
28 - Yeoh
29 - Aldo
30 - Cuarón
31 - Angus
32 - Addy
33 - Godard
34 - Pitt
35 - Shimamoto
36 - Srinath
37 - Geoffrey
38 - Jonze
39 - Farida
40 - Piggy
41 - Meurisse
42 - Jackson
43 - Yamadera
44 - Aitana
45 - Weaving
46 - Nina
47 - Paxton
48 - Parambrata
49 - Maurier
50 - Srinidhi
51 - Zarah
52 - Gondry
53 - Tosar
54 - Gulaczyk
55 - Gayatri
56 - Harry
57 - Tigmanshu
58 - Burtt
59 - Christine
60 - Romano
61 - Farhan
62 - Robert
63 - Moira
64 - Bull
65 - Pannu
66 - Cortaz
67 - Sylvia
68 - Frost
69 - Mastrantonio
70 - Audley
71 - Groucho
72 - Kareena
73 - Cab
74 - Danieli
75 - Lanchester
76 - Heston
77 - Dandotiya
78 - Sakshi
79 - Dorsey
80 - Pawan
81 - Temuera
82 - Blethyn
